In [ ]:
import polars as pl
import sys
import os
import re
from types import SimpleNamespace

sys.path.append(os.path.dirname(os.path.abspath('')))

from libraries.client_stashapp import get_stashapp_client, StashAppClient

stash = get_stashapp_client()
stash_client = StashAppClient()

def get_tags_by_names(tag_names, stash_client=stash):
    """
    Convenient function to get multiple tags by name and return them as named variables.
    
    Args:
        tag_names: List of tag name strings
        stash_client: The stash client instance (defaults to global stash)
    
    Returns:
        SimpleNamespace object with tag names as clean variable names
    """
    tags = SimpleNamespace()
    
    print("🏷️  Tag Lookup Results:")
    print("=" * 50)
    
    for tag_name in tag_names:
        # Create clean variable name
        var_name = re.sub(r'[:\s\.\-\(\)]', '_', tag_name.lower())
        var_name = re.sub(r'_+', '_', var_name)  # Replace multiple underscores with single
        var_name = var_name.strip('_')  # Remove leading/trailing underscores
        
        try:
            tag = stash_client.find_tag(tag_name)
            if tag:
                setattr(tags, var_name, tag)
                print(f"✅ {var_name:30} -> {tag_name} (ID: {tag.get('id', 'N/A')})")
            else:
                setattr(tags, var_name, None)
                print(f"❌ {var_name:30} -> {tag_name} (NOT FOUND)")
        except Exception as e:
            setattr(tags, var_name, None)
            print(f"⚠️  {var_name:30} -> {tag_name} (ERROR: {str(e)})")
    
    print("=" * 50)
    print(f"Found {sum(1 for attr in dir(tags) if not attr.startswith('_') and getattr(tags, attr) is not None)} out of {len(tag_names)} tags")
    
    return tags

def update_marker_tags(marker, new_tag_id):
    existing_tags = marker["tags"]
    existing_tags_ids = [tag["id"] for tag in existing_tags]
    updated_tags_ids = existing_tags_ids + [new_tag_id]
    print(marker["scene"]["title"])
    print(existing_tags_ids, "->", updated_tags_ids)
    stash.update_scene_marker({
        "id": marker["id"],
        "tag_ids": updated_tags_ids
    })

In [ ]:
# Get multiple tags conveniently
marker_tag_names = [
    "AI",
    "Marker Status: Confirmed",
    "Marker Source: PySceneDetect",
    "Marker Source: Skier AI",
    "Marker Source: Manual", 
    "Marker Source: timestamp.trade",
    "Marker Source: TPDB"
]

marker_tags = get_tags_by_names(marker_tag_names)

# Confirmed markers with unknown source

In [ ]:
confirmed_markers_with_unknown_source = stash.find_scene_markers(
    {
        "tags": {
            "value": [marker_tags.marker_status_confirmed["id"]],
            "modifier": "INCLUDES_ALL",
            "excludes": [marker_tags.marker_source_manual["id"], marker_tags.marker_source_timestamp_trade["id"], marker_tags.marker_source_tpdb["id"], marker_tags.marker_source_skier_ai["id"]],
        }
    },
    fragment="id primary_tag { id name } tags { id name } scene { id title }",
)
confirmed_markers_with_unknown_source

In [ ]:
for confirmed_marker in confirmed_markers_with_unknown_source:
    existing_tags = confirmed_marker["tags"]
    existing_tags_ids = [tag["id"] for tag in existing_tags]
    updated_tags_ids = existing_tags_ids + [marker_tags.marker_source_skier_ai["id"]]
    print(confirmed_marker["scene"]["title"])
    print(updated_tags_ids)
    stash.update_scene_marker({
        "id": confirmed_marker["id"],
        "tag_ids": updated_tags_ids
    })

# Markers with AI tags with unknown source

In [ ]:
ai_markers_with_unknown_source = stash.find_scene_markers(
    {
        "tags": {
            "value": [marker_tags.ai["id"]],
            "depth": 1,
            "modifier": "INCLUDES_ALL",
            "excludes": [
                marker_tags.marker_status_confirmed["id"],
                marker_tags.marker_source_manual["id"],
                marker_tags.marker_source_timestamp_trade["id"],
                marker_tags.marker_source_tpdb["id"],
                marker_tags.marker_source_pyscenedetect["id"],
                marker_tags.marker_source_manual["id"],
                marker_tags.marker_source_skier_ai["id"],
            ],
        }
    },
    filter = {"per_page": 10000},
    fragment="id primary_tag { id name } tags { id name } scene { id title }",
)
ai_markers_with_unknown_source[0:5]

In [ ]:
for marker in ai_markers_with_unknown_source:
    update_marker_tags(marker, marker_tags.marker_source_skier_ai["id"])